In [1]:
import sys
import cv2
import numpy as np
from rasterio import Affine
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from extract.extract import get_coordinates
from extract.extract import read_image
from matplotlib import pyplot as plt
from rasterio.shutil import copy
from collections import namedtuple
from swak.funcflow import Pipe, Map
from transform import gcp_to_rasterio, AsGeoTif, get_transformation
import folium


In [2]:
IMAGE_PATH = '../../resources/us-army-1945.jpeg'
TEST_TIF_PATH = "/Users/viktor/Work/8gradplus/time-strolls/resources/tifs/test.tif"

# lat / lon convention!
# refactor [{pixel, gps}]
Point = namedtuple('Point', ['x', 'y'])

# lat / lon convention - assume input taken from google 
GROUND_CONTROL_POINTS = [
    {
        'info': 'Kirche St. Oswald',          
        'pixel': Point(286, 2125),
        'gps': Point(48.619095, 14.030765), 
        'id': None,
        'zoom': None,
    },
    {
        'info': 'Laher Unteruresch', 
        'pixel': Point(863, 2610) , 
        'gps': Point(48.61017854015886, 14.04406485511563) ,
        'id': None,
        'zoom': None,
    },
    {
        'info': 'Ruine Wittinghausen',
        'pixel': Point(2919, 143),
        'gps': Point(48.64500581431426, 14.103290101060226),
        'id': None,
        'zoom': None,
    }, 
]

In [3]:
def show_rasterio(tif_path):
    with rasterio.open(tif_path) as src:
        fig, ax = plt.subplots()
        rasterio.plot.show(src, ax)


def show_tiles(path=TEST_TIF_PATH):
    """Needs locally running tile server (make run-api. in backend folder)"""
    
    tiles = f"http://127.0.0.1:8000/tiles/WebMercatorQuad/{'{z}'}/{'{x}'}/{'{y}'}.png?url=file:///{path}"
    center = [48.61017, 14.044]  # Example: Berlin, Germany
    
    m = folium.Map(
        location=center,
        zoom_start=16,
        tiles=None  # Disable default tiles
    )
    
    
    folium.TileLayer(
        tiles=tiles,
        attr='historic',
        name="1945",
    ).add_to(m)
    
    return m

In [4]:
img = read_image(IMAGE_PATH)
affine = Pipe(Map(gcp_to_rasterio), get_transformation)(GROUND_CONTROL_POINTS)
AsGeoTif(img, TEST_TIF_PATH)(affine)
show_tiles(TEST_TIF_PATH)